# Лабораторная работа №2


_ПИН-212 Жан-Лин Никита_

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
# Импортируем функции для разделения данных на обучающую и тестовую выборки,для поиска гиперпараметров модели
from sklearn.model_selection import train_test_split, GridSearchCV
# Импортируем классы для стандартизации признаков
from sklearn.preprocessing import StandardScaler
# Импортируем функции для оценки качества модели
from sklearn.metrics import classification_report, accuracy_score
# Импортируем классификатор опорных векторов (SVM)
from sklearn.svm import SVC

In [3]:
# Читаем данные из CSV-файла
data = pd.read_csv('smoking_driking_dataset_Ver03.csv', encoding='utf-8')
print(data.head(5))

    sex  age  height  weight  waistline  sight_left  sight_right  hear_left  \
0  Male   35     170      75       90.0         1.0          1.0        1.0   
1  Male   30     180      80       89.0         0.9          1.2        1.0   
2  Male   40     165      75       91.0         1.2          1.5        1.0   
3  Male   50     175      80       91.0         1.5          1.2        1.0   
4  Male   50     165      60       80.0         1.0          1.2        1.0   

   hear_right    SBP  ...  LDL_chole  triglyceride  hemoglobin  urine_protein  \
0         1.0  120.0  ...      126.0          92.0        17.1            1.0   
1         1.0  130.0  ...      148.0         121.0        15.8            1.0   
2         1.0  120.0  ...       74.0         104.0        15.8            1.0   
3         1.0  145.0  ...      104.0         106.0        17.6            1.0   
4         1.0  138.0  ...      117.0         104.0        13.8            1.0   

   serum_creatinine  SGOT_AST  SGOT_AL

In [5]:
# Преобразуем 'sex' в числовую категорию
data['sex'] = data['sex'].map({'Male': 1, 'Female': 0}).astype(int)

# Преобразуем 'DRK_YN' в бинарные метки
data['DRK_YN'] = data['DRK_YN'].map({'Y': 1, 'N': 0}).astype(int)

# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())

Уникальные значения в data['DRK_YN']: [1 0]


In [7]:
columns_to_drop = [
    'SMK_stat_type_cd',
    'sight_left', 'sight_right',  # Если зрение с обеих сторон не добавляет дополнительной информации
    'hear_left', 'hear_right',  # Если слух с обеих сторон не добавляет дополнительной информации
    'serum_creatinine', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP'  
]

# Удаление столбцов
data = data.drop(columns=columns_to_drop, axis=1)

# Проверка результата
print(data.head())


   sex  age  height  weight  waistline    SBP   DBP   BLDS  tot_chole  \
0    1   35     170      75       90.0  120.0  80.0   99.0      193.0   
1    1   30     180      80       89.0  130.0  82.0  106.0      228.0   
2    1   40     165      75       91.0  120.0  70.0   98.0      136.0   
3    1   50     175      80       91.0  145.0  87.0   95.0      201.0   
4    1   50     165      60       80.0  138.0  82.0  101.0      199.0   

   HDL_chole  LDL_chole  triglyceride  hemoglobin  urine_protein  DRK_YN  
0       48.0      126.0          92.0        17.1            1.0       1  
1       55.0      148.0         121.0        15.8            1.0       0  
2       41.0       74.0         104.0        15.8            1.0       0  
3       76.0      104.0         106.0        17.6            1.0       0  
4       61.0      117.0         104.0        13.8            1.0       0  


In [9]:
# Проверка уникальных значений в целевой переменной
print("Уникальные значения в data['DRK_YN']:", data['DRK_YN'].unique())


Уникальные значения в data['DRK_YN']: [1 0]


In [11]:
# Разделение данных на признаки и целевую переменную
X = data.drop('DRK_YN', axis=1)
Y = data['DRK_YN']

In [13]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = X.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
X[numeric_features] = scaler.fit_transform(X[numeric_features])

In [15]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [17]:
# Создание модели SVM
svm = SVC()

# Определение сетки параметров для поиска оптимальных гиперпараметров
parameters = {
    'kernel': ['rbf', 'linear', 'sigmoid'],
    'C': [0.1, 1],  
    'gamma': [0.1, 1]  
}

# Создание объекта GridSearchCV для поиска оптимальных гиперпараметров модели
grid_search = GridSearchCV(svm, parameters, cv=3, verbose=3, n_jobs=-1)

# Запуск поиска оптимальных гиперпараметров на обучающем наборе данных
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1], 'gamma': [0.1, 1],
                         'kernel': ['rbf', 'linear', 'sigmoid']},
             verbose=3)

In [19]:
print("Лучшие параметры:", grid_search.best_params_)

Лучшие параметры: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


In [21]:
# Обучение модели SVM с лучшими параметрами
best_svm = grid_search.best_estimator_
best_svm.fit(X_train, Y_train)

SVC(C=1, gamma=0.1)

In [23]:
Y_pred_svm = best_svm.predict(X_test)


In [25]:
# Создаем отчет о классификации
report_svm = classification_report(Y_test, Y_pred_svm)
print(report_svm)

accuracy = accuracy_score(Y_test, Y_pred_svm)
print("Оценка модели:", accuracy)

              precision    recall  f1-score   support

           0       0.73      0.67      0.70      1216
           1       0.69      0.74      0.71      1184

    accuracy                           0.71      2400
   macro avg       0.71      0.71      0.71      2400
weighted avg       0.71      0.71      0.71      2400

Оценка модели: 0.7054166666666667
